# CDAN Model Evaluation on Real Test Set

Evaluate CDAN trained model for sim2real adaptation.

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from torch.autograd import Function

In [ ]:
# Paths & params
REAL_TEST_DIR = 'data/real/test'
MODEL_PATH = 'model/cdan_resnet18.pth'
BATCH_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Data Loader
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
test_dataset = datasets.ImageFolder(REAL_TEST_DIR, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
classes = test_dataset.classes

In [ ]:
class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None
def grad_reverse(x, alpha):
    return GradReverse.apply(x, alpha)

class CDANModel(nn.Module):
    def __init__(self, num_classes):
        super(CDANModel,self).__init__()
        self.feature = models.resnet18(pretrained=True)
        self.feature.fc = nn.Identity()
        self.class_classifier = nn.Linear(512,num_classes)
        self.domain_classifier = nn.Sequential(
            nn.Linear(512*num_classes, 100),
            nn.ReLU(),
            nn.Linear(100,2)
        )
    def forward(self,x,alpha=0):
        feat = self.feature(x)
        class_output = self.class_classifier(feat)
        softmax_out = torch.softmax(class_output, dim=1)
        op = torch.bmm(softmax_out.unsqueeze(2), feat.unsqueeze(1))
        op = op.view(op.size(0), -1)
        reverse_feat = grad_reverse(op, alpha)
        domain_output = self.domain_classifier(reverse_feat)
        return class_output, domain_output

In [ ]:
model = CDANModel(len(classes)).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

In [ ]:
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs, _ = model(inputs)
        _, preds = torch.max(outputs,1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

accuracy = np.mean(all_preds==all_labels)
print(f'CDAN Accuracy on Real Test Set: {accuracy:.4f}')

report = classification_report(all_labels, all_preds, target_names=classes)
print('Classification Report:\n', report)

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - CDAN on Real Test Set')
plt.show()